In [1]:
import pandas as pd
import googlemaps
import json

import sys
sys.path.append('..')
from secret import google_api_key

## Table Analysis

In [2]:
data = pd.read_csv('data/20년1차_청년매입임대_주택내역(정정)_수정편집.csv', encoding='cp949')
print(data.shape, end=' -> ')
data = data[data['지역본부']== '서울지역본부']
data = data[data['성별전용']!= '여성전용']
data = data[data['공급형']== '단독(1인)']
data = data.iloc[:, 3:]
print(data.shape)
data.head(3)

(1161, 16) -> (361, 13)


,관리부서,명,주소,호수,전용\r\n면적(㎡),주거공용\r\n면적(㎡),면적계(㎡),공급형,성별전용,임대보증금(원),월임대료(원),주택열람일정,관리비
0,서울중부권주거복지지사,매입다가구(서울중구),서울특별시 중구 마른내로 106(오장동),202,20.81,17.42,38.23,단독(1인),남여공용,산정중,산정중,열람불가(계약전 주택개방),NaN
1,서울중부권주거복지지사,매입다가구(서울중구),서울특별시 중구 마른내로 106(오장동),203,21.17,17.75,38.92,단독(1인),남여공용,산정중,산정중,열람불가(계약전 주택개방),NaN
2,서울중부권주거복지지사,매입다가구(서울중구),서울특별시 중구 마른내로 106(오장동),204,20.81,17.42,38.23,단독(1인),남여공용,산정중,산정중,열람불가(계약전 주택개방),NaN


In [3]:
addr_count = data['주소'].value_counts()
addr_list = [item.replace('美家','미가').replace('(',' ').replace(')','')[6:] for item in addr_count.index]
print()
print(f'# of unique locations : {len(addr_count.index)}\n')

for count, addr in zip(addr_count.values, addr_list):
    print(f'{count:>2} = {addr}')


# of unique locations : 35

85 = 동대문구 이문로 17 회기동 
58 = 노원구 동일로174길 38 공릉동 플라체76
50 = 중구 마른내로 106 오장동 
26 = 영등포구 영등포로49길 9-14 영등포동2가 101동
22 = 광진구 능동로39길 5 중곡동 와이위드미
14 = 중랑구 사가정로 374 면목동,킴스타워 637-2외 4필지
13 = 강서구 양천로26길 33-8 방화동 마곡스카이 A단지 101동
 9 = 광진구 자양번영로7길 12 자양동 위너시원
 8 = 광진구 동일로80길 37 중곡동 더 메인즈빌
 8 = 강북구 삼양로138길 54 수유동 스카이드림타운
 6 = 양천구 목동중앙서로11길 51 목동 형지리버빌
 4 = 양천구 목동중앙남로16다길 32-7 목동 엘림하우스
 4 = 강북구 삼양로163길 27 우이동 
 4 = 영등포구 영등포로49길 9-14 영등포동2가 102동
 4 = 강북구 삼양로114길 3 수유동,은행빌라 은행빌라
 4 = 강북구 덕릉로30길 49 미아동,청운미가 102동
 4 = 강북구 덕릉로30길 49 미아동,청운미가 103동
 4 = 강북구 덕릉로30길 49 미아동,청운미가 101동
 3 = 강북구 삼양로38나길 7 미아동,프랑지파니 B동 프랑지파니 B동
 3 = 노원구 동일로237나길 19 상계동,시온4동 미소4동
 3 = 성북구 보국문로8가길 7 정릉동 반석영우빌
 3 = 서대문구 증가로19길 7 북가좌동 지우팰리스
 3 = 서초구 서초동 1584-6,11,12 
 3 = 성북구 돌곶이로9가길 26-7 석관동 다우아트빌
 3 = 노원구 동일로237나길 17 상계동,시온3동 해밀2동
 2 = 구로구 구로동로7길 25 가리봉동 103동
 2 = 은평구 은평로13길 11-14 녹번동 나린하우징5차 가동
 2 = 은평구 역말로6길 6-14 역촌동 동산주택홈타운
 1 = 도봉구 도봉로117길 51-14 쌍문동 스마트힐
 1 = 성북구 화랑로33가길 29 장위동 
 1 = 성북구 장월로6길 11 장위동 하이

## Geocoding by googlemaps

In [4]:
gmaps = googlemaps.Client(key=google_api_key)
gmaps

In [5]:
geocode_list = [gmaps.geocode(item, language='ko')[0] for item in addr_list]
for tmp_dict, tmp_addr in zip(geocode_list, addr_list):
    tmp_dict['original_addr'] = tmp_addr

addr_company = '영등포구 여의도동 14-35 금강빌딩'
geocode_company = gmaps.geocode(addr_company, language='ko')[0]
geocode_company['original_addr'] = addr_company
geocode_company

{'address_components': [{'long_name': '１４−３５',
   'short_name': '１４−３５',
   'types': ['premise']},
  {'long_name': '여의도동',
   'short_name': '여의도동',
   'types': ['political', 'sublocality', 'sublocality_level_2']},
  {'long_name': '영등포구',
   'short_name': '영등포구',
   'types': ['political', 'sublocality', 'sublocality_level_1']},
  {'long_name': '서울특별시',
   'short_name': '서울특별시',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': '대한민국', 'short_name': 'KR', 'types': ['country', 'political']},
  {'long_name': '150-010', 'short_name': '150-010', 'types': ['postal_code']}],
 'formatted_address': '대한민국 서울특별시 영등포구 여의도동 14-35',
 'geometry': {'location': {'lat': 37.5285662, 'lng': 126.9206151},
  'location_type': 'ROOFTOP',
  'viewport': {'northeast': {'lat': 37.5299151802915,
    'lng': 126.9219640802915},
   'southwest': {'lat': 37.5272172197085, 'lng': 126.9192661197085}}},
 'place_id': 'ChIJ60ZLwCSffDURVzGbMD-vD7U',
 'plus_code': {'compound_code': 'GWHC+C6 대한민국 서울특별시'

In [6]:
geocode_list.insert(0, geocode_company)
r_json = json.dumps(geocode_list, indent=4)

with open('seoul_rental_houses.json', 'w') as fp:
    fp.write(r_json)